In [61]:
import pandas as pd

In [62]:
data = pd.read_csv('data/hour.csv')

In [63]:
data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [64]:
data.drop('instant', axis=1, inplace=True)

In [65]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dteday      17379 non-null  object 
 1   season      17379 non-null  int64  
 2   yr          17379 non-null  int64  
 3   mnth        17379 non-null  int64  
 4   hr          17379 non-null  int64  
 5   holiday     17379 non-null  int64  
 6   weekday     17379 non-null  int64  
 7   workingday  17379 non-null  int64  
 8   weathersit  17379 non-null  int64  
 9   temp        17379 non-null  float64
 10  atemp       17379 non-null  float64
 11  hum         17379 non-null  float64
 12  windspeed   17379 non-null  float64
 13  casual      17379 non-null  int64  
 14  registered  17379 non-null  int64  
 15  cnt         17379 non-null  int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 2.1+ MB


In [66]:
data.describe()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,2.501640,0.502561,6.537775,11.546752,0.028770,3.003683,0.682721,1.425283,0.496987,0.475775,0.627229,0.190098,35.676218,153.786869,189.463088
std,1.106918,0.500008,3.438776,6.914405,0.167165,2.005771,0.465431,0.639357,0.192556,0.171850,0.192930,0.122340,49.305030,151.357286,181.387599
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.480000,0.104500,4.000000,34.000000,40.000000
50%,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.630000,0.194000,17.000000,115.000000,142.000000
75%,3.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,220.000000,281.000000
max,4.000000,1.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,367.000000,886.000000,977.000000


In [67]:
data.replace({'weekday':{1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5:'Friday',6:'Saturday',7:'Sunday'}}, inplace=True)
# data.replace({'mnth':{1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'June',7:'July',8:'Aug',9:'Sept',10:'Oct',11:'Nov',12:'Dec'}}, inplace=True)
data.replace({'season': {1:'Spring', 2:'Summer', 3:'Fall', 4:'Winter'}}, inplace=True)
data.replace({'yr': {0:'2011', 1:'2012'}}, inplace=True)


In [68]:
data.head(5)

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,Spring,2011,1,0,0,Saturday,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01,Spring,2011,1,1,0,Saturday,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01,Spring,2011,1,2,0,Saturday,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01,Spring,2011,1,3,0,Saturday,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01,Spring,2011,1,4,0,Saturday,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [69]:
data.groupby(['season']).sum()

,mnth,hr,holiday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
season,,,,,,,,,,,,
Fall,34573,51738,96,3141,5981,3176.02,2949.3918,2846.72,771.4810,226091,835038,1061129
Spring,13234,49413,164,2794,6194,1268.98,1264.6101,2466.08,912.4850,60622,410726,471348
Summer,20520,50757,96,3066,6365,2401.42,2295.0915,2764.54,896.8359,203522,715067,918589
Winter,45293,48763,144,2864,6230,1790.72,1759.4021,2823.27,722.9044,129782,711831,841613


In [70]:
data.groupby(['yr']).sum()

,mnth,hr,holiday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
yr,,,,,,,,,,,,
2011,56832,100054,239,5911,12428,4228.00,4054.5089,5562.45,1652.6842,247252,995851,1243103
2012,56788,100617,261,5954,12342,4409.14,4213.9866,5338.16,1651.0221,372765,1676811,2049576


In [71]:
data.groupby(['mnth']).sum()

,hr,holiday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
mnth,,,,,,,,,,,
1,16715,71,907,2082,339.60,343.9261,830.15,296.5903,12042,122891,134933
2,15639,47,913,1957,402.58,403.1323,760.63,289.6041,14963,136389,151352
3,17060,0,1069,2172,575.80,563.1266,867.67,327.7673,44444,184476,228920
4,16550,48,957,2093,675.42,656.8665,845.12,336.7225,60802,208292,269094
5,17112,48,1032,2175,885.06,841.7050,1025.17,272.2874,75285,256401,331686
6,16560,0,1032,1859,985.10,919.0738,829.16,267.0047,73906,272436,346342
7,17112,48,984,1866,1124.14,1047.6917,889.64,247.0956,78157,266791,344948
8,16968,0,1104,1951,1045.34,961.0454,939.91,252.9160,72039,279155,351194
9,16554,48,957,2139,885.92,832.5180,1026.67,238.3354,70323,275668,345991
